In [1]:
# Necessary library and module imports
from jupyter_dash import JupyterDash
import dash
from dash import Dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from crud_module import AnimalShelter



###########################
# Data Manipulation / Model
###########################

# Credentials for database access
username = "aacuser"
password = "EUbqdVfypi"

# Create instance of AnimalShelter with the given credentials
shelter = AnimalShelter(username, password)

# Convert records from the database into a DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Drop unnecessary column '_id'
df.drop(columns=['_id'],inplace=True)



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Load and encode logo image for display
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout of the dashboard
app.layout = html.Div([
    html.Div(
        style={'display': 'flex', 'align-items': 'center', 'justify-content': 'center'},
        children=[
            html.A(
                href="http://www.snhu.edu",
                children=[
                    html.Img(
                        src='data:image/png;base64,{}'.format(encoded_image.decode()),
                        style={
                            'height': '100px',
                            'margin': '10px'
                        }
                    )
                ],
                target="_blank"
            ),
            html.B(html.H1('SNHU CS-340 Dashboard - Matthew Tyson'))
        ]
    ),
    
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness', 'value': 'Mountain or Wilderness'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'reset-button'}
            ],
            value=None,
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[0],

        # Enabling filtering and sorting
        filter_action="native",
        sort_action="native",

        # Pagination settings
        page_action="native",
        page_current=0,
        page_size=10,

        # Styling
        style_table={'height': '500px', 'overflowY': 'auto'},
        style_cell={'minWidth': '95px', 'width': '95px', 'maxWidth': '95px',
                    'whiteSpace': 'normal'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        }
    ),
    html.Br(),
    html.Hr(),
    
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the dashboard based on the selected filter
@app.callback(
    [Output('datatable-id','data'), 
     Output('filter-type', 'value')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Define logic to filter data based on selected filter type
    
    if filter_type == 'reset-button':
        query = {}
    
    elif filter_type == "Water Rescue":
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        
    elif filter_type == "Mountain or Wilderness":
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
        
    elif filter_type == "Disaster or Individual Tracking":
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    else:
        query = {}
    
    filtered_data = shelter.read(query)
    df_filtered = pd.DataFrame.from_records(filtered_data)
    df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records'), dash.no_update

# Callback to update the pie chart based on the data in the table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Logic to generate a pie chart based on the breed column in the table data
    
    if viewData is None:
        return dcc.Graph()
    dff = pd.DataFrame.from_dict(viewData)
    
    breed_counts = dff['breed'].value_counts()
    
    top_9_breeds = breed_counts.head(9).index.tolist()
    
    dff['breed'] = dff['breed'].apply(lambda breed: breed if breed in top_9_breeds else 'Other')
    
    return [
        dcc.Graph(            
            figure=px.pie(dff, names='breed')
        )
    ]

# Callback to update the styling of selected columns in the table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Logic to highlight the selected columns
    
    if selected_columns is None:
        return []
    
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Callback to update the map based on the data in the table and the selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Logic to display a map marker based on the selected row in the table  
    
    if viewData is None:
        return html.Div("No data to display")
    
    dff = pd.DataFrame.from_dict(viewData)
    
    if dff.empty or len(dff.columns) < 15:
        return html.Div("Insufficient data to display map")
    
    row = 0 if index is None or not index else index[0]

    if row >= len(dff):
        return html.Div("Selected row index is out of range")
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48], zoom=10, children=[
               dl.TileLayer(id="base-layer-id"),
               dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                          children=[
                              dl.Tooltip(dff.iloc[row, 4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row, 9])
                              ])
                          ])
               ])
    ]

# Run the app
app.run_server(mode="external")

/home/matthewtyson_snhu/.local/lib/python3.9/site-packages/dash/dash.py:525: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:20071/
